# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [2]:
#загружаем библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
#загружаем файл
data=pd.read_csv('/datasets/insurance.csv')

In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные представлены в удобном формате, пропусков нет, предобработки не требуется.

In [6]:
# разделим целевой признак и признаки
target = data['Страховые выплаты']
features = data.drop('Страховые выплаты', axis=1)

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [31]:
# создадим матрицу
X = np.array([[1,5,6,3],
             [1,1,1,2],
             [1,2,2,4],
            [1,4,2,2],
            [1,5,1,1],
             [1,2,3,1]]
            )

P=np.array([[5,2,2,2],
             [1,1,1,1],
             [3,4,2,3],
           [2,3,1,1]])

# проверка обратимости (питон выдаст ошибку если необратимая)
invert = np.linalg.inv(P)

#зададим величину сдвига/целевой признак
y=np.array([3,1,3,4,2,1])

#найдем вектора весов исходной
w=np.linalg.inv(X.T@X)@X.T@y

print(X@w)

[3.29718712 1.35507542 3.1112923  2.71667346 2.67672238 0.84304933]


In [32]:
#найдем вектор весов для матрицы помноженной на обратную
XP=X@P
w_n=np.linalg.inv(XP.T@XP)@XP.T@y
print(XP@w_n)

[3.29718712 1.35507542 3.1112923  2.71667346 2.67672238 0.84304933]


**Ответ:** Если исходную матрицу умножать на обратимую качество модели линейной регрессии не изменится.

**Обоснование:** Выше представлены примеры, показывают при перемножении исходной матрицы X на случайную обратную матрицу P, предсказания модели остануться неизменными. Матрица XP будет иметь свой вектор весов, который учтем изменные признаки и выдаст аналогичное предсказание.$$a =Xw =XPw_n$$

## Алгоритм преобразования

**Алгоритм**

- $F$ — матрица признаков 

- $y$ — вектор целевого признака

- $w$ — вектор весов линейной регрессии 

- $F'$ — помноженная матрица признаков F (на случайную обратимую матрицу P)

- $a'$ — предсказания помноженной матрицы

- $w'$ — вектор весов в случае помноженной матрицы



Алгоритм будет заключаться в том чтобы перемножать матрицу признаков на случайную обратимую матрицу, что позволит защищитить данные и не изменит качества модели линейной регрессии.

$$a=Fw=a'=F'w'$$



**Обоснование**

Раскладываем формулу вектора с помощью $(AB)^{-1} = B^{-1}A^{-1}$ и $(AB)^{T} = B^TA^T$:

Рассмотрим новый вектор весов и через него выделим алгоритм:
$$w'= ((FP)^T FP)^{-1} FP^T y = P^{-1} (F^T F)^{-1} (P^T)^{-1} P^TF^Ty$$

Транспонированая матрица P^T при умножении (P^T)^{-1} даст единицу убираем их из формулы:
$$P^{-1} (F^TF)^{-1}  F^T y$$
    
Если в эту формулу подставить обычный вектор то у нас выходит:
$$P^{-1}(F^T F)^{-1} F^T y = P^{-1}w$$

Теперь подставим в формулу предсказания:
$$a'= F'P^{-1}w= FPP^{-1}w =Fw$$
    


## Проверка алгоритма

In [9]:
# проверяем алгоритм на моделе линейной регрессии сначала по исходным данным посчитаем метрику r2
model = LinearRegression()
model.fit(features,target)
print(model.score(features,target))

0.42494550286668


Метрика r2 модели на исходных данных. Проверим на измененных признаках.

In [10]:
#создадим матрицу размера признаков из случайных чисел и перемножим
P=np.random.randint(1,100, size = (4, 4))
new_features=features@P
new_features

,0,1,2,3
0,4167370.0,2728186.0,1538478.0,4567234.0
1,3192989.0,2090149.0,1178963.0,3500491.0
2,1764609.0,1155058.0,651580.0,1934784.0
3,3503287.0,2293656.0,1293206.0,3838566.0
4,2193074.0,1435603.0,809675.0,2403911.0
...,...,...,...,...
4995,2999434.0,1963670.0,1107346.0,3287238.0
4996,4402337.0,2882125.0,1625123.0,4824139.0
4997,2848066.0,1864654.0,1051386.0,3120870.0
4998,2747417.0,1798762.0,1014284.0,3010760.0


In [11]:
model.fit(new_features,target)
print(model.score(new_features,target))

0.4249455028666468


Алгоритм реализован и полностью себе оправдал. Метрики вышли одинаковые в обоих случаях. Умножение на рандомную обратимую матрицу помогло скрыть данные клиентов и в тоже время сохранить корректность работы модели.